# Estimating the aDDM

Author: Brenden Eum (2023)

This script will read already-transformed data containing trial-by-trial details and fixations and fit the aDDM to this data. It uses the Tavares Toolbox (Tavares et al. 2017) rewritten for Julia by Lynn in Summer 2023. See https://github.com/aDDM-Toolbox/ADDM.jl for the toolbox. 

I've made a changes to the toolbox in order to suit my project. Some of these are small changes, like changing object types, saving outputs to csv files, or changing the grid for grid search. I don't document small changes since they'll probably change over multiple iterations. I do document major changes to the toolbox: (1) The toolbox now fits the model to the output of load_data_from_csv instead of simulating data; (2) The toolbox no longer throws a domain error when $\theta \notin [0,1]$.

Inputs for this notebook:
- Data
    - data/expdata*.csv:    Data pertaining to the details of each trial. An observation is a subject-trial. It must include the following columns [parcode,trial,rt,choice,item_left,item_right], which correspond to [subject, trial, response time in ms, choice (-1 left, 1 right), value of left item, value of right item] respectively.
    - data/fixations*.csv:  Data pertaining to the fixations within each trial. An observation is a subject-trial-fixation. It must include columns [parcode,trial,fix_item,fix_time], which correspond to [subject, trial, fixation location (1 left, 2 right, otherwise other), fixation duration in ms].
- Code
    - src/ addm_grid_search.jl: Takes a grid of parameters, makes every combination, and calculates negative log likelihood (NLL) of aDDM for every combo given some data.
    - addm.jl: Defines aDDM class and functions for calculating trial likelihood, summing likelihoods, and simulating.
    - ddm.jl: Defines DDM class and functions for calculating trial likelihood, summing likelihoods, and simulating.
    - util.jl: Contains functions for prepping data for aDDM fitting and estimating empirical fixation properties.


Outputs for this notebook:
- GainFit.csv:  aDDM estimates for the gain condition. Estimates are for subjects included in the input datasets.
- LossFit.csv:  aDDM estimates for the loss condition.

# Preamble

A set of standard things I like to do before getting started with any notebook.

In [1]:
import Pkg;
Pkg.add("DataFrames");
Pkg.add("CSV");
using DataFrames;
using CSV;

┌ Warning: could not download https://pkg.julialang.org/registries
│   exception = RequestError: Could not resolve host: pkg.julialang.org while requesting https://pkg.julialang.org/registries
└ @ Pkg.Registry /opt/homebrew/Cellar/julia/1.9.3/share/julia/stdlib/v1.9/Pkg/src/Registry/Registry.jl:69
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


# Activate the Environment

Typically this happens by

```
using Pkg
Pkg.activate("addm")
```

**Note that single vs. double quotes matter.**

But this does not work as expected in a notebook unless the `Project.toml` is in the same folder as the notebook file.

In [2]:
pwd();
Base.load_path();
include("../../src/addm_grid_search.jl");
Base.load_path();

  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`
  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`
  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`
  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`
  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`
  Activating project at `~/Toolboxes/ADDM.jl/docs/src/addm`


# Prepping to Fit the aDDM

Let's do a 2-iteration grid search. We'll start with a low resolution grid search, then once we have rough estimates per subject, we can zoom into the region around the rough estimates for a precise measure.

## Prep for gain and loss: load data, placeholders, set step sizes

In [3]:
addm = aDDM(.005, .07, .3); # These can be anything. They exist because you need an aDDM object. (d, s, t).

dataGain = load_data_from_csv("../../data/expdataGain.csv", "../../data/fixationsGain.csv", convertItemValues=nothing)
dataLoss = load_data_from_csv("../../data/expdataLoss.csv", "../../data/fixationsLoss.csv", convertItemValues=nothing);

subjectCount = length(collect(keys(dataGain)))

pChangeNLLThreshold = 0.001 # How little should NLL change in order for us to stop zooming in with grid search? 0.1%

dStepSize = .0025 # Step sizes for the grids.
σStepSize = .03
θStepSize = .5
bStepSize = .25

dInitialGrid = Any[]
σInitialGrid = Any[]
θInitialGrid = Any[]
bInitialGrid = Any[]
for subject in collect(keys(dataGain))
    push!(dInitialGrid, [0.0001:dStepSize:.008;])   # d from .0001 to .008
    push!(σInitialGrid, [0.01:σStepSize:.1;])      # σ from .01 to .1
    push!(θInitialGrid, float([0:θStepSize:2;]))    # θ from 0 to 2
    push!(bInitialGrid, [-.25:bStepSize:.25;])      # b from -.25 to .25
end

println(dInitialGrid[1])
println(σInitialGrid[1])
println(θInitialGrid[1])
println(bInitialGrid[1])

[0.0001, 0.0026, 0.0051, 0.0076]
[0.01, 0.04, 0.07, 0.1]
[0.0, 0.5, 1.0, 1.5, 2.0]
[-0.25, 0.0, 0.25]


## Function: Grid search and report data frame

In [4]:
function fit_aDDM_custom_resolution(addm::aDDM, data::Dict{String, Vector{aDDMTrial}}, dGrid::Vector{Any}, σGrid::Vector{Any}, θGrid::Vector{Any}, bGrid::Vector{Any}, subjectCount::Number)
    """
    """

    #print("Approx how many likelihoods to calculate per subject: ")
    #print(length(dGrid[1])*length(σGrid[1])*length(θGrid[1])*length(bGrid[1]))

    dList = Vector{Float64}(undef, subjectCount)
    σList = Vector{Float64}(undef, subjectCount)
    θList = Vector{Float64}(undef, subjectCount)
    bList = Vector{Float64}(undef, subjectCount)
    NLLsList = Vector{Float64}(undef, subjectCount);

    ind = 1
    @showprogress for subject in collect(keys(data))

        dEst, σEst, θEst, bEst, NLL_Indiv = aDDM_grid_search(addm, data, dGrid[ind], σGrid[ind], θGrid[ind], bGrid[ind], subject)
    
        dList[ind] = dEst[1]
        σList[ind] = σEst[1]
        θList[ind] = θEst[1]
        bList[ind] = bEst[1]
        NLLsList[ind] = NLL_Indiv
        ind += 1
    
    end

    df = DataFrame(
        subject = collect(keys(data)),
        d = dList,
        s = σList,
        t = θList,
        b = bList
    )
    df = sort(df, :subject)
    NLL = sum(NLLsList)
    return df, NLL
end

fit_aDDM_custom_resolution (generic function with 1 method)

## Function: Make new grid

In [5]:
function make_new_grid(estimates::DataFrame, data::Dict{String, Vector{aDDMTrial}}, dStepSize::Number, σStepSize::Number, θStepSize::Number, bStepSize::Number, iteration::Number)
    
    dGrid = Any[]
    σGrid = Any[]
    θGrid = Any[]
    bGrid = Any[]

    iteration = iteration-1 # Iteration in the while loops below start at 2 since we run the fitting once before the loop. Correct for this.
    ind = 1
    for subject in collect(keys(data))

        d = estimates[(estimates.subject .== subject), "d"][1]
        σ = estimates[(estimates.subject .== subject), "s"][1]
        θ = estimates[(estimates.subject .== subject), "t"][1]
        b = estimates[(estimates.subject .== subject), "b"][1]

        dLow = max(0.0001, d-(dStepSize/(2^iteration)));
        dHigh = d+(dStepSize/(2^iteration));
        σLow = max(0.001, σ-(σStepSize/(2^iteration)));
        σHigh = σ+(σStepSize/(2^iteration));
        θLow = θ-(θStepSize/(2^iteration));
        θHigh = θ+(θStepSize/(2^iteration));
        bLow = max(-.99, b-(bStepSize/(2^iteration)));
        bHigh = min(.99, b+(bStepSize/(2^iteration)));

        push!(dGrid, float([dLow, d, dHigh]))
        push!(σGrid, float([σLow, σ, σHigh]))    
        push!(θGrid, float([θLow, θ, θHigh]))           
        push!(bGrid, float([bLow, b, bHigh]))  

        ind += 1
    end
    return dGrid, σGrid, θGrid, bGrid
end

make_new_grid (generic function with 1 method)

# Fit Gain Data

In [6]:
oldGainEstimates, oldNLL = fit_aDDM_custom_resolution(addm, dataGain, dInitialGrid, σInitialGrid, θInitialGrid, bInitialGrid, subjectCount)
println("Iteration 1"); print("Old NLL missing"); print("New NLL "); println(oldNLL); print("Percent Change missing"); println(oldGainEstimates);

iteration = 2
Δ = 100
while Δ > .001
    
    dGrid, σGrid, θGrid, bGrid = make_new_grid(oldGainEstimates, dataGain, dStepSize, σStepSize, θStepSize, bStepSize, iteration)
    newGainEstimates, newNLL = fit_aDDM_custom_resolution(addm, dataGain, dGrid, σGrid, θGrid, bGrid, subjectCount)
    Δ = (oldNLL-newNLL)/oldNLL
    if Δ < 0
        global gainEstimates = oldGainEstimates
    else
        global gainEstimates = newGainEstimates
    end

    print("Iteration "); println(iteration); print("Old NLL "); println(oldNLL); print("New NLL "); println(newNLL); print("Percent Change "); println(Δ); println(gainEstimates); iteration += 1; oldGainEstimates = newGainEstimates; oldNLL = newNLL;

end
CSV.write("GainFit.csv", gainEstimates)

Progress: 100%|█████████████████████████████████████████| Time: 0:05:52


Iteration 1
Old NLL missingNew NLL 13445.629063325236
Percent Change missing26×5 DataFrame
 Row │ subject  d        s        t        b       
     │ String   Float64  Float64  Float64  Float64 
─────┼─────────────────────────────────────────────
   1 │ 201       0.0026     0.07      0.5      0.0
   2 │ 202       0.0026     0.04      0.5      0.0
   3 │ 204       0.0026     0.04      1.0      0.0
   4 │ 205       0.0026     0.07      0.5      0.0
   5 │ 206       0.0026     0.07      1.0      0.0
   6 │ 207       0.0026     0.04      0.5      0.0
   7 │ 208       0.0051     0.07      1.0      0.0
   8 │ 209       0.0026     0.04      0.5      0.0
   9 │ 211       0.0026     0.04      1.0      0.0
  10 │ 213       0.0026     0.1       0.5      0.0
  11 │ 214       0.0026     0.04      0.5      0.0
  12 │ 215       0.0026     0.04      1.0      0.0
  13 │ 216       0.0051     0.07      0.5      0.0
  14 │ 217       0.0026     0.04      0.5      0.0
  15 │ 219       0.0026     0.07      1

Progress: 100%|█████████████████████████████████████████| Time: 0:01:57


Iteration 2
Old NLL 13445.629063325236
New NLL 13211.89798758597
Percent Change 0.017383424355859933
26×5 DataFrame
 Row │ subject  d        s        t        b       
     │ String   Float64  Float64  Float64  Float64 
─────┼─────────────────────────────────────────────
   1 │ 201      0.0026     0.055     0.5     0.0
   2 │ 202      0.0026     0.04      0.5     0.125
   3 │ 204      0.0026     0.055     0.75    0.0
   4 │ 205      0.0026     0.07      0.5     0.0
   5 │ 206      0.00385    0.055     0.75    0.0
   6 │ 207      0.00135    0.055     0.75    0.0
   7 │ 208      0.0051     0.07      0.75    0.0
   8 │ 209      0.00135    0.055     0.75    0.0
   9 │ 211      0.0026     0.04      0.75    0.0
  10 │ 213      0.0026     0.085     0.25    0.0
  11 │ 214      0.0026     0.04      0.75    0.125
  12 │ 215      0.0026     0.055     0.75    0.0
  13 │ 216      0.00385    0.055     0.75   -0.125
  14 │ 217      0.00135    0.055     0.75    0.0
  15 │ 219      0.00135    0.055    

Progress: 100%|█████████████████████████████████████████| Time: 0:01:57


...

Progress: 100%|█████████████████████████████████████████| Time: 0:01:57


Iteration 4
Old NLL 13096.966240349335
New NLL 13073.326908041005
Percent Change 0.0018049471820047798
26×5 DataFrame
 Row │ subject  d          s        t        b        
     │ String   Float64    Float64  Float64  Float64  
─────┼────────────────────────────────────────────────
   1 │ 201      0.0026     0.05125   0.625   -0.03125
   2 │ 202      0.0029125  0.04375   0.5625   0.0625
   3 │ 204      0.0026     0.04375   0.8125  -0.03125
   4 │ 205      0.0022875  0.0625    0.5     -0.03125
   5 │ 206      0.00385    0.0625    0.8125  -0.03125
   6 │ 207      0.0016625  0.04375   0.625   -0.03125
   7 │ 208      0.004475   0.07375   0.875   -0.03125
   8 │ 209      0.0016625  0.0475    0.8125  -0.03125
   9 │ 211      0.0022875  0.04375   0.75    -0.03125
  10 │ 213      0.0029125  0.085     0.375   -0.03125
  11 │ 214      0.0022875  0.04375   0.625    0.0625
  12 │ 215      0.0022875  0.0475    0.8125  -0.03125
  13 │ 216      0.0041625  0.05125   0.6875  -0.125
  14 │ 217      0.0

Progress: 100%|█████████████████████████████████████████| Time: 0:01:57


Iteration 5
Old NLL 13073.326908041005
New NLL 13066.484954537422
Percent Change 0.0005233521315354316
26×5 DataFrame
 Row │ subject  d           s         t        b         
     │ String   Float64     Float64   Float64  Float64   
─────┼───────────────────────────────────────────────────
   1 │ 201      0.00244375  0.05125   0.65625  -0.046875
   2 │ 202      0.0029125   0.041875  0.53125   0.0625
   3 │ 204      0.00244375  0.045625  0.84375  -0.046875
   4 │ 205      0.00213125  0.0625    0.5      -0.046875
   5 │ 206      0.00385     0.060625  0.8125   -0.046875
   6 │ 207      0.00150625  0.045625  0.65625  -0.046875
   7 │ 208      0.00463125  0.07375   0.875    -0.046875
   8 │ 209      0.0016625   0.0475    0.8125   -0.046875
   9 │ 211      0.00244375  0.041875  0.75     -0.046875
  10 │ 213      0.00306875  0.085     0.375    -0.046875
  11 │ 214      0.0022875   0.04375   0.625     0.0625
  12 │ 215      0.00244375  0.045625  0.8125   -0.046875
  13 │ 216      0.00431875  

"GainFit.csv"

# Fit Loss Data

In [7]:
oldLossEstimates, oldNLL = fit_aDDM_custom_resolution(addm, dataLoss, dInitialGrid, σInitialGrid, θInitialGrid, bInitialGrid, subjectCount)
println("Iteration 1"); print("Old NLL missing"); print("New NLL "); println(oldNLL); print("Percent Change missing"); println(oldLossEstimates);

iteration = 2
Δ = 100
while Δ > .001
    
    dGrid, σGrid, θGrid, bGrid = make_new_grid(oldLossEstimates, dataLoss, dStepSize, σStepSize, θStepSize, bStepSize, iteration)
    newLossEstimates, newNLL = fit_aDDM_custom_resolution(addm, dataLoss, dGrid, σGrid, θGrid, bGrid, subjectCount)
    Δ = (oldNLL-newNLL)/oldNLL
    if Δ < 0
        global lossEstimates = oldLossEstimates
    else
        global lossEstimates = newLossEstimates
    end

    print("Iteration "); println(iteration); print("Old NLL "); println(oldNLL); print("New NLL "); println(newNLL); print("Percent Change "); println(Δ); println(lossEstimates); iteration += 1; oldLossEstimates = newLossEstimates; oldNLL = newNLL; 

end
CSV.write("LossFit.csv", lossEstimates)

Progress: 100%|█████████████████████████████████████████| Time: 0:06:51


Iteration 1
Old NLL missingNew NLL 14208.625775361294
Percent Change missing26×5 DataFrame
 Row │ subject  d        s        t        b       
     │ String   Float64  Float64  Float64  Float64 
─────┼─────────────────────────────────────────────
   1 │ 201       0.0001     0.07      2.0     0.0
   2 │ 202       0.0026     0.04      1.0     0.0
   3 │ 204       0.0026     0.04      1.0     0.0
   4 │ 205       0.0001     0.07      2.0     0.0
   5 │ 206       0.0026     0.07      1.0     0.0
   6 │ 207       0.0026     0.04      0.5     0.0
   7 │ 208       0.0026     0.07      1.5     0.0
   8 │ 209       0.0026     0.04      1.0     0.0
   9 │ 211       0.0026     0.04      1.0     0.0
  10 │ 213       0.0026     0.07      1.5     0.0
  11 │ 214       0.0001     0.07      2.0     0.0
  12 │ 215       0.0026     0.04      0.5    -0.25
  13 │ 216       0.0026     0.07      1.5     0.0
  14 │ 217       0.0026     0.04      0.5     0.0
  15 │ 219       0.0026     0.07      1.5     0.0
  

Progress: 100%|█████████████████████████████████████████| Time: 0:02:18


Iteration 2
Old NLL 14208.625775361294
New NLL 13863.818348713245
Percent Change 0.024267471893444335
26×5 DataFrame
 Row │ subject  d        s        t        b       
     │ String   Float64  Float64  Float64  Float64 
─────┼─────────────────────────────────────────────
   1 │ 201      0.00135    0.055     1.75    0.0
   2 │ 202      0.0026     0.04      1.0     0.0
   3 │ 204      0.00135    0.055     1.25    0.0
   4 │ 205      0.0001     0.07      2.25    0.0
   5 │ 206      0.0026     0.055     1.0     0.125
   6 │ 207      0.00135    0.04      0.75    0.0
   7 │ 208      0.0026     0.07      1.75    0.0
   8 │ 209      0.00135    0.04      1.25    0.0
   9 │ 211      0.00135    0.04      1.25    0.0
  10 │ 213      0.0026     0.07      1.25    0.125
  11 │ 214      0.0001     0.055     2.25    0.0
  12 │ 215      0.00135    0.04      0.75   -0.125
  13 │ 216      0.0026     0.07      1.25    0.0
  14 │ 217      0.00135    0.04      0.75   -0.125
  15 │ 219      0.0026     0.055 

LoadError: UndefVarError: `newGainEstimates` not defined